<a href="https://colab.research.google.com/github/A-l-E-v/ML-Engineer/blob/main/request_3_6_%D0%BA%D0%BE%D0%BD%D1%81%D0%BF%D0%B5%D0%BA%D1%82.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Парсинг веб-сайтов

## Часть 2. Парсинг множества страниц (crawling)

Часто бывает, что нужно достать информацию не с одной страницы, а с многих. В таком случае нужно написать скрипт, который будет обходить нужные нам страницы и парсить их. Такой скрипт еще иногда называют кроулером (crawler).

Попробуем распарсить Форум на КиноПоиске, а именно раздел "Фильмы ХХI века": https://forumkinopoisk.ru/forumdisplay.php?f=5  
Пусть наша задача звучит так: собрать названия тем, количество ответов, количество просмотров, ссылку на тред.

Прежде чем начать парсинг, убедимся, что сайт вообще возможно получить обычным запросом.

In [ ]:
# Импортируем необходимые библиотеки
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
# Задаем URL на forumkinopoisk
URL= "https://forumkinopoisk.ru/forumdisplay.php?f=5"

In [ ]:
# Отправляем GET-запрос по указанному URL и выводим статус код ответа
response = requests.get(URL)
response.status_code

SSLError: HTTPSConnectionPool(host='forumkinopoisk.ru', port=443): Max retries exceeded with url: /forumdisplay.php?f=5 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:997)')))

Видим ошибку SSLError. Это относительно легкая проблема, и она исправляется отключением проверки SSL-сертификата. Мы не будем подробно останаливаться на том, что это такое и зачем нужно - просто запомните, что в таких случаях проверку надо отключать.

In [ ]:
# Отправляем GET-запрос, но отключим проверку SSL-сертификата
response = requests.get(URL, verify=False)
response.status_code

C:\ProgramData\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'forumkinopoisk.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


200

Видим предупреждение от интерпретатора. Иногда они сигнализируют о важных, но пока не критичных проблемах. Однако сейчас они будут только мешаться.

In [ ]:
#  отключим предупреждения("варнинги") для всего проекта
import warnings
warnings.filterwarnings("ignore")

Приступим к парсингу сайта.  
Выпишем несколько страниц из этого раздела:  
https://forumkinopoisk.ru/forumdisplay.php?f=5&order=desc  
https://forumkinopoisk.ru/forumdisplay.php?f=5&order=desc&page=2  
...  
https://forumkinopoisk.ru/forumdisplay.php?f=5&order=desc&page=13  
https://forumkinopoisk.ru/forumdisplay.php?f=5&order=desc&page=14  
...  
https://forumkinopoisk.ru/forumdisplay.php?f=5&order=desc&page=464  
https://forumkinopoisk.ru/forumdisplay.php?f=5&order=desc&page=465    
Видим, что нумерация страниц на сайте и в url-адресе совпадают. У первой ссылки номера страницы нет, но если его подставить, то ссылка работает: https://forumkinopoisk.ru/forumdisplay.php?f=5&order=desc&page=1

In [ ]:
# Напишем функцию для создания списка всех страниц
def make_page_list(base_URL, max_number):
    page_list=[]
    for i in range(1, max_number+1):
        page_list.append(base_URL+ '&page='+ str(i))
    return page_list

In [ ]:
# Создаем список страниц с использованием функции make_page_list
page_list = make_page_list('https://forumkinopoisk.ru/forumdisplay.php?f=5&order=desc', 465)
# Выводим первые 5 страниц и последнюю страницу из списка
page_list[:5], page_list[-1]

(['https://forumkinopoisk.ru/forumdisplay.php?f=5&order=desc&page=1',
  'https://forumkinopoisk.ru/forumdisplay.php?f=5&order=desc&page=2',
  'https://forumkinopoisk.ru/forumdisplay.php?f=5&order=desc&page=3',
  'https://forumkinopoisk.ru/forumdisplay.php?f=5&order=desc&page=4',
  'https://forumkinopoisk.ru/forumdisplay.php?f=5&order=desc&page=5'],
 'https://forumkinopoisk.ru/forumdisplay.php?f=5&order=desc&page=465')

### Парсинг первой страницы

Сначала разберемся как все работает на примере одной страницы.

In [ ]:
# Напишем функцию для отправки get-запроса и превращения его в soup
def get_soup(URL):
    response = requests.get(URL, verify=False)
    soup = BeautifulSoup(response.text, 'lxml')
    return soup

In [ ]:
# Получим soup для первой страницы из списка page_list и извлекаем текстовое содержимое страницы
soup = get_soup(page_list[0])
soup.text

C:\ProgramData\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'forumkinopoisk.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nФильмы ХХI века - Форум на КиноПоиске\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\xa0\nФорум на КиноПоиске\n> О фильмах\n\n\n\n \r\n\tФильмы ХХI века\r\n\r\n\n\n\n\n\n\n\n\n\n\nИмя\n\nЗапомнить?\n\n\nПароль\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nСправка\nПользователи\nСоциальные группы\nКалендарь\nПоиск\nСообщения за день\nВсе разделы прочитаны\n\n\n\n\n\n\n\nФильмы ХХI века Художественные и документальные, детские и взрослые, хорошие и плохие, цветные и черно-белые. Один фильм - одна тема.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nСтраница 1 из 465\n1\n231151101\n>\nПоследняя »\n\n\n\n\n\n\n\n\nТемы раздела : Фильмы ХХI века\nОпции раздела \nИскать в этом разделе\n\n\n\n\n\n\n\n\nПросмотров: 1,064 \nОбъявление: Премия ФКП: Топ 100 лучших фильмов ужасов, Премия ФКП 1931: Выбор форума [лауреаты]\n\n\n31.07.2023\n\nOrientir\r\n\t\t\t\t(Firefly)\r\n\t\t\t\n\n\n\n\n\xa0\n\nТема  /\r\n\t\tАвтор\n\nПоследнее 

In [ ]:
# Находим элемент таблицы с помощью BeautifulSoup на основе указанных атрибутов
table = soup.find('tbody', attrs = {"id": "threadbits_forum_5"})

In [ ]:
# Находим все ячейки в указанной строке таблицы
cells = table.find_all('tr')[5].find_all('td')

In [ ]:
# Возвращаем третью ячейку (с индексом 2) из списка ячеек
cells[2]

<td class="alt1" id="td_threadtitle_280664" title="Опасный друг || Good Boy  
 
https://i.ibb.co/72MNzsC/4373222-imdb-logo-logos-icon.png 
 
https://i.ibb.co/2PCYm2C/W1500-51871763.jpg 
 
Сюжет:">
<div>
<span style="float:right">
<a href="#" onclick="attachments(280664); return false"> <img alt="Вложений: 4" border="0" class="inlineimg" src="images/misc/paperclip.gif"/></a>
</span>
<a href="showthread.php?s=0e59d367e69d8e505515fa4dc9284de3&amp;t=280664" id="thread_title_280664">Опасный друг (Good Boy)</a>
</div>
<div class="smallfont">
<span onclick="window.open('member.php?s=0e59d367e69d8e505515fa4dc9284de3&amp;u=732960', '_self')" style="cursor:pointer">Psyhoze</span>
</div>
</td>

Видим, что в ячейке в перемешку лежит много разной информации.

In [ ]:
#  рассмотрим только элемент содержащий название треда
cells[2].find_all('a')[1]

<a href="showthread.php?s=0e59d367e69d8e505515fa4dc9284de3&amp;t=280664" id="thread_title_280664">Опасный друг (Good Boy)</a>

In [ ]:
# пройдемся по всем тегам <a> и обработаем только то, где есть аттрибут id
for a in cells[2].find_all('a'):
    if 'id' in a.attrs:
        name = a.text
        thread_url = a['href']
        break
print(name, thread_url)

Опасный друг (Good Boy) showthread.php?s=0e59d367e69d8e505515fa4dc9284de3&t=280664


Если мы зайдем на тред через сайт, то увидим, что ссылка выглядит так https://forumkinopoisk.ru/showthread.php?t=280664.   
Однако ссылка, полученная парсингом и прибавлением адреса сайта (https://forumkinopoisk.ru/), https://forumkinopoisk.ru/showthread.php?s=0e59d367e69d8e505515fa4dc9284de3&t=280664 - тоже рабочая.

In [ ]:
#  С остальными данными все просто
cells[4].text, cells[5].text

('8', '1,002')

Оформим все это в функции

In [ ]:
# Функция парсинга ячейки с именем
def parse_name_cell(cell):
    for a in cell.find_all('a'):
        if 'id' in a.attrs:
            name = a.text
            thread_url = "https://forumkinopoisk.ru/" + a['href']
            return (name, thread_url)

In [ ]:
# Функция парсинга одной таблицы
def parse_table(table):
    content=[]
    for row in table.find_all('tr'):
        if not row.find_all('th'):
            try:
                cells = row.find_all('td')
                name, thread_url = parse_name_cell(cells[2])
                if name == "Правила раздела":
                    continue
                answer_amount = cells[4].text
                views_amount = cells[5].text
                content.append([name, thread_url, answer_amount, views_amount])
            except (IndexError, TypeError, KeyError):
                pass
    table_df  = pd.DataFrame(content, columns=["name", "thread_url", "answer_amount", "views_amount"])
    return table_df

In [ ]:
#  оформим результат в таблицу
table_df = parse_table(table)
table_df.head()

,name,thread_url,answer_amount,views_amount
0,Хэллоуин заканчивается (Halloween Ends),https://forumkinopoisk.ru/showthread.php?s=0e5...,116,"22,072"
1,Человек-паук: Нет пути домой (Spider-Man: No W...,https://forumkinopoisk.ru/showthread.php?s=0e5...,"8,770","1,151,565"
2,Отряд самоубийц (Suicide Squad),https://forumkinopoisk.ru/showthread.php?s=0e5...,"8,440","1,413,478"
3,Фредди против Джейсона (Freddy vs. Jason),https://forumkinopoisk.ru/showthread.php?s=0e5...,"1,526","162,029"
4,Опасный друг (Good Boy),https://forumkinopoisk.ru/showthread.php?s=0e5...,8,"1,002"


### Парсинг всех таблиц

In [ ]:
# импортируем библиотеку для прогресс-бара
from tqdm import tqdm

tqdm библиотека нужна для удобного остлеживания прогресса.

In [ ]:
#  импортируем библиотеку времени
import time

In [ ]:
# посмотрим как работает sleep таймер
print("hello")
time.sleep(5)
print("world")

hello
world


Часто в сайты встроена защита от кроулинга: если слишком много зарпосов подряд идет с одного кмопьютера, то все его запросы будут блокироваться. Это исправляется добавлением sleep таймера между отправкой запросов. Это действие решает проблему с блокировкой, но значительно замедляет сбор ифнормации. В данном случае сайт не имет подобной защиты, поэтому мы не будем использовать таймер.

In [ ]:
# пройдемся по всем старницам, распарсим таблицы, и добавим их к финальному дадасету
big_df = pd.DataFrame()
for page in tqdm(page_list):
    soup = get_soup(page)
    table = soup.find('tbody', attrs = {"id": "threadbits_forum_5"})
    table_df = parse_table(table)
    big_df = pd.concat([big_df, table_df], ignore_index=True)
    # time.sleep(5)

100%|██████████| 465/465 [01:35<00:00,  4.85it/s]


In [ ]:
# посмотрим на результат
big_df

,name,thread_url,answer_amount,views_amount
0,Ты ранил мои чувства (You Hurt My Feelings),https://forumkinopoisk.ru/showthread.php?s=c20...,2,544
1,Хэллоуин заканчивается (Halloween Ends),https://forumkinopoisk.ru/showthread.php?s=c20...,116,"22,100"
2,Человек-паук: Нет пути домой (Spider-Man: No W...,https://forumkinopoisk.ru/showthread.php?s=c20...,"8,770","1,151,595"
3,Отряд самоубийц (Suicide Squad),https://forumkinopoisk.ru/showthread.php?s=c20...,"8,440","1,413,498"
4,Фредди против Джейсона (Freddy vs. Jason),https://forumkinopoisk.ru/showthread.php?s=c20...,"1,526","162,050"
...,...,...,...,...
25,Воспитатели (Die Fetten Jahre sind vorbei),https://forumkinopoisk.ru/showthread.php?s=c20...,11,"4,706"
26,U2 в 3D (U2 3D),https://forumkinopoisk.ru/showthread.php?s=c20...,13,"5,270"
27,Грайндхаус (Grind House),https://forumkinopoisk.ru/showthread.php?s=c20...,413,"47,517"
28,Титаник 2 (Jack is back),https://forumkinopoisk.ru/showthread.php?s=c20...,15,"11,971"


Подведем итоги. В ходе урока мы научились обходить сайт и парсить содержимое большого количества страниц.